### Assignment 10 Question 1

In [60]:
class Transaction:
    '''
    Fields
    amount (Float)
    date (Str)
    transaction_type (Str), 
    description (Str),
    category (Str)
    Requires:
     - transaction_type is either "income" or "expense".
     - date is in the format "YYYY-MM-DD"
     - whenever the transaction is an "expense" category is one of: 
      "Entertainment", "Home", "Transportation", "Food", "Health", "Bills", 
      "Electronics", "Miscellaneous"
      otherwise category is the empty string
    '''    
    def __init__(self, amount, date, transaction_type, description,category):
        '''
        Constructor: creates a Transaction object with all fields matching the 
        given parameters
        
        __init__: Transaction, Float, Str, Str, Str, Str
        Requires: see field requirements above
        
        Effects: Mutates self
        
        '''
        
        self.amount = amount
        self.date = date
        self.transaction_type = transaction_type
        self.description = description
        self.category = category
    

    def __repr__(self):
        '''
        Returns a string representation of self
        
        (Implicitly called by print(t), where t is of type Transaction)
        
        __repr__: Transaction -> Str
        
        '''
        
        if self.category == '':
            category_display = 'N/A'
        else:
            category_display = self.category

        s = "Transaction | Date: {0}, Type: {1}, Amount: ${2:,.2f}, Desc: {3}, Category: {4}"
        
        return s.format(self.date, self.transaction_type.capitalize(), 
                        self.amount, self.description, category_display)


    def __eq__(self, other):
        '''
        Returns True if self and other share the same Transaction attributes.
        
        __eq__: Transaction Any -> Bool

        '''
        return (isinstance(other, Transaction) and
                self.amount == other.amount and
                self.description == other.description and
                self.date == other.date and
                self.transaction_type == other.transaction_type and
                self.category == other.category)

def crop(descript):
    if descript[-1] == '}':
        desc = descript[:-1]
    elif descript[-2:] == '}\n':
        desc = descript[:-2]
    else:
        desc = descript
    return desc
 
def process_transactions(filename):
    fin = open(filename)
    txns = list(fin.readlines())
    txn_objects = []
    transactions = []
    for txn in txns:
        objs = txn.split(',')
        txn_objects.append(objs)
    for txn in txn_objects:
        amt = float(txn[0].split(' ')[1])
        date = txn[1].split(' ')[-1]
        descript = txn[3].split(':')[1][1:]
        desc = crop(descript)
        if txn[2].split(' ')[2] == 'expense':
            txn_type = 'expense'
            category = txn[-1].split(' ')[2]
            category = crop(category)
        else:
            txn_type = 'income'
            category = ""
        transaction = Transaction(amt, date, txn_type, desc, category)
        transactions.append(transaction)
    return transactions

# Tests
# (see testing note above regarding testing with files on EdX)
# basic.txt contains a single transaction 
# {amount: 100.0, date: 2000-01-01, transaction_type: income, description: Salary}
# check.expect("MarkUs Basic Test",process_transactions("basic.txt"),[Transaction(100.0,"2000-01-01","income","Salary","")])
process_transactions("basic.txt")

[Transaction | Date: 2000-01-01, Type: Income, Amount: $100.00, Desc: Salary, Category: N/A,
 Transaction | Date: 2000-01-04, Type: Expense, Amount: $50.00, Desc: Electrical Bill, Category: Bills]

### Assignment 10 Question 2

In [24]:


class Account:
    '''
    Fields:
    owner (Str)
    transactions (listof Transaction)
    '''    
    
    def __init__(self, owner):
        '''
        Constructor: creates an Account object for owner with an empty 
        transactions list
        
        __init__: Transaction, Str
        
        Effects: Mutates self
        
        '''        
       
        self.owner = owner
        self.transactions = []
        
    def __repr__(self):
        '''
        Returns a string representation of self
        
        (Implicitly called by print(t), where t is of type Account)
        
        __repr__: Account -> Str
        
        '''
        s = "Account(owner={0}, balance={1})"
        
        return s.format(self.owner, self.get_balance())
    
    def __eq__(self,other):
        '''
        Returns True if self and other share the same owner and transactions.
        
        __eq__: Account Any -> Bool

        '''
        return (isinstance(other, Account) and
                self.owner == other.owner and
                self.transactions == other.transactions) 
                # this will call the __eq__ function of Transaction class 

    def add_transaction(self, transaction):
        '''
        Adds a new transaction to the account.
        
        Effects:
        Mutates self

        add_transaction: Transaction -> None
        '''
        self.transactions.append(transaction)

    def list_transactions(self):
        '''
        Returns a list of all transactions in self sorted by date

        list_transactions: Account -> (lisof Transaction)
        '''
        return sorted(self.transactions,key = numeric_date)
    

    def filter_type_by_amount(self,ttype,threshold):
        '''
        Returns a list of all transacations in self that are of type ttype whose
        amount value is above or equal to threshold, sorted by date.
        
        filter_type_by_amount: Account Str Float -> (listof Transaction)
        '''
        L = self.transactions
        ts = list(filter(lambda t: t.transaction_type == ttype,L))
        sortTs = list(filter(lambda t: t.amount >= threshold,ts))
        return sorted(sortTs, key = numeric_date)        
    
   
    def filter_expenses_by_category(self, category):
        '''
        Returns a list of all transactions in self of type "expense" 
        that match category, sorted by date
        
        filter_expenses_by_category: Account Str -> (listof Transaction)
        Requires:
        category is one of:" Entertainment", "Home", "Transportation", 
        "Food", "Health", "Bills", "Electronics", "Miscellaneous"
        
        '''
        ts = list(filter(lambda t: t.category == category,self.transactions))
        return sorted(ts, key = numeric_date)
    
    
    def filter_transactions_by_date_range(self,start_date,end_date):
        '''
        Returns a list of all transactions in self whose date is on or after 
        start_date and whose date is on or before end_date, sorted by date
        
        filter_transactions_by_date_range: Account Str Str
        Requires:
        start_date and end_date are in the format "YYYY-MM-DD"
        
        '''
        L = []
        tempTransLB = Transaction(0,start_date,'','','')
        tempTransUB = Transaction(0,end_date,'','','')
        
        
        for t in self.transactions:
            low_num = numeric_date(tempTransLB)
            curr_num = numeric_date(t)
            high_num = numeric_date(tempTransUB)
            if  low_num <= curr_num  <= high_num:
                L.append(t)
                
        return sorted(L,key = numeric_date)
    
        
        
    def get_balance(self):
        ''' 
        Returns the balance in the Account self
        
        get_balance: Account -> Float
        
        Example:
        if self.transactions = 
        [Transaction(200.0, "2024-01-02", "expense", "Groceries" ,"Food"),         
         Transaction(150.0, "1995-06-24", "income", "Birthday", "")]
        then self.get_balance() => -50.0
        '''
        ## YOUR CODE HERE
        pass
          
### end of Account class ###

def numeric_date(T):
    '''
    Returns the day since the date 1-1-1 that a transcation T occurred 
    (assuming 365 days every year, with 366 days every 4 years)
    
    numeric_date: Transaction -> Nat
    
    Examples:
    if T = Transaction(0,'1-1-5','income','','') then numeric_date(T) => 5
    if T = Transaction(0, '2-2-1','income','','') then numeric_date(T) => 397
    '''
    #date = T.date.strip('"')
    values = T.date.split("-")
    year = int(values[0])
    months = int(values[1])
    days = int(values[2])
    
    
    daysPerYear = 365
    extraDays = year // 4
    
    daysInYearFromMonths = 0
    for month in range(1,months):
        if month in [1,3,5,7,8,10,12]: #shouldn't technically reach 12
            daysInYearFromMonths += 31
        elif month == 2:
            daysInYearFromMonths += 28
        else:
            daysInYearFromMonths += 30
        
    
    return daysPerYear*(year-1)+ daysInYearFromMonths + days + extraDays       


    
def display_transactions(tList):
    '''
    Prints all transactions in tList, one per line with a final 
    blank line at the end.
    
    Effects: prints to screen

    display_all_transactions: (listof Transaction) -> None
    
    Examples:
    if tList = [Transaction(200.0, "2024-01-02", "expense", "Groceries" ,"Food"),         
                Transaction(150.0, "1995-06-24", "income", "Birthday", "")]
    then display_transactions(tList) => None and prints
    Transaction | Date: 2024-01-02, Type: Expense, Amount: $200.00, Desc: Groceries, Category: Food
    Transaction | Date: 1995-06-24, Type: Income, Amount: $150.00, Desc: Birthday, Category: N/A
    [empty line here]
    '''   
    for transaction in tList:
      print(transaction, end = "\n")

tList = [Transaction(200.0, "2024-01-02", "expense", "Groceries" ,"Food"), Transaction(150.0, "1995-06-24", "income", "Birthday", "")]
display_transactions(tList)

Transaction | Date: 2024-01-02, Type: Expense, Amount: $200.00, Desc: Groceries, Category: Food
Transaction | Date: 1995-06-24, Type: Income, Amount: $150.00, Desc: Birthday, Category: N/A
